# Sparse tensors

A new functionality of Tensor Fox is the support for sparse tensors. Let $T \in \mathbb{R}^{I_1 \times \ldots \times I_L}$ be a sparse tensor with *nnz* nonzero entries. The tensor is represented as a triple *[data, idxs, dims]*, where *data* is an array of size $nnz$ such that *data[i]* is the $i$-th nonzero entry of $T$, with corresponding index *idxs[i]*. It is necessary to pass *idxs* also as an array, which will be of shape $nnz \times L$. Finally, we have that *dims* $= [I_1, \ldots, I_L]$. 

In [1]:
import numpy as np
import TensorFox as tfx
import matplotlib.pyplot as plt

## Small example

Below we start with a fourth order sparse tensor $10 \times 10 \times 10 \times 10$ with rank $R = 5$ such that each factor matrix has 5 nonzero entries. We can generate sparse factor matrices with the function *gen_rand_sparse_tensor* with inputs **dims**, **R**, **nnz** as showed below. We remark that the sparsity of the tensor is not necessarily equal to the sparsity of its factor matrices.

In [2]:
# Dimensions of the tensor.
dims = [10, 10, 10, 10]

# Rank of the tensor.
R = 5

# Number of nonzero entries of each factor matrix. 
nnz = 10

# Generate sparse tensor.
print('Factor matrix nonzero entries =', round(100 * nnz / np.prod(dims[0] * R), 2), '%')
data, idxs, dims, factors = tfx.gen_rand_sparse_tensor(dims, R, nnz)
print('Tensor nonzero entries =', round(100 * len(data) / np.prod(dims), 2), '%')

# Define sparse tensor.
T = [data, idxs, dims]

Factor matrix nonzero entries = 20.0 %
Tensor nonzero entries = 12.96 %


Here we prefer the damped Gauss-Newton method to compute the CPD, but the Tensor Train CPD also works. We remark that it is not possible to use $\verb|tol| \_ \verb|mlsvd| = -1$. In the case the user sets $\verb|display| = 3$ or $4$, the program computes only the error associated with the nonzero entries, otherwise we would face memory issues. 

In [3]:
class options:
    display = 1
    method = 'dGN'
    
factors, output = tfx.cpd(T, R, options)

Sparse tensor detected
    nnz = 1296
    Sparsity level = 87.04 %
-----------------------------------------------------------------------------------------------
Computing MLSVD
    Compressing unfolding mode 1
    Compressing unfolding mode 2
    Compressing unfolding mode 3
    Compressing unfolding mode 4
    Compression detected
    Compressing from (10, 10, 10, 10) to (5, 5, 5, 5)
-----------------------------------------------------------------------------------------------
Type of initialization: random
-----------------------------------------------------------------------------------------------
Computing CPD
Final results
    Number of steps = 85
    Relative error = 8.36877284601443e-15
    Accuracy =  100.0 %


The relative error showed above only take in account the nonzero entries of $T$ (the same is valid for the compression error which is equal to zero in this example). The factor matrices of the decomposition may introduce small errors when approximating the zeros, and these small errors summed together does increase the actual error of the CPD. Thus the relative error showed above is a lower bound to the actual error, but usually it is close enough.

Since this is a small example we can put everything in dense format and verify what is the actual error. However this won't be possible for really large tensors. Regardless, the main point of the CPD is to approximate the nonzero entries, which is always accomplished.

In [4]:
# Generate the coordinate (dense) format of the approximation.
T_approx = tfx.cpd2tens(factors)

# Generate the dense format from the sparse representation.
T_dense = tfx.sparse2dense(data, idxs, dims)

# Compute the error.
np.linalg.norm(T_dense - T_approx)/np.linalg.norm(T_dense)

8.376276617764773e-15

## Memory cost

There is a big reduction in memory cost when working with sparse representations. Below we show a graph with the maximum memory cost attained in the computation of the CPDs of sparse $n \times n \times n$ tensors (blue curve) vs. the cost to store these tensors in dense format. As we can see, the difference is substantial, Tensor Fox does avoid the intermediate memory explosion problem. For instance, the sparse approach requires $3648$ megabytes when $n = 30000$, whereas the dense approach requires approximately $205$ terabytes. 

![sparse](sparse.png)

## Big example

The amazing memory reduction showed above is due to the fact that Tensor Fox needs $\mathcal{0}(I_\ell^2)$ flops of memory for each unfolding whereas the other classic methods needs $\mathcal{O}(\prod_{k \neq \ell} I_k)$ flops. Still, one can face tensor with dimensions at the order of billions or more, and in these cases even Tensor Fox will blow up the memory. To address this issue one can "divide" the dimensions in smaller pieces and then compute a CPD in a space with more dimensions, but lower ones. This approach is computationally feasible and can find good solutions. For more detail about this approach see (). 

In the example we generate a fourth order tensor $10^3 \times 10^3 \times 10^3 \times 10^3$ with rank $R = 5$. 

In [5]:
# Dimensions of the tensor.
dims = [10**3, 10**3, 10**3, 10**3]

# Rank of the tensor.
R = 5

# Number of nonzero entries of each factor matrix. 
nnz = 10

# Generate sparse tensor.
print('Factor matrix nonzero entries =', round(100 * nnz / np.prod(dims[0] * R), 6), '%')
data, idxs, dims, factors = tfx.gen_rand_sparse_tensor(dims, R, nnz)
print('Tensor nonzero entries =', 100 * len(data) / np.prod(dims, dtype=np.uint64), '%')

# Define sparse tensor.
T = [data, idxs, dims]

Factor matrix nonzero entries = 0.2 %
Tensor nonzero entries = 1e-06 %


In [6]:
options.display = 1
factors, output = tfx.cpd(T, R, options)

Sparse tensor detected
    nnz = 10000
    Sparsity level = 99.999999 %
-----------------------------------------------------------------------------------------------
Computing MLSVD
    Compressing unfolding mode 1
    Compressing unfolding mode 2
    Compressing unfolding mode 3
    Compressing unfolding mode 4
    Compression detected
    Compressing from (1000, 1000, 1000, 1000) to (5, 5, 5, 5)
-----------------------------------------------------------------------------------------------
Type of initialization: random
-----------------------------------------------------------------------------------------------
Computing CPD
Final results
    Number of steps = 200
    Relative error = 1.3728911506304369e-14
    Accuracy =  100.0 %


We finish a few observations about the parameter *mkl_dot*. By default the program uses a specific MKL sparse dot function to perform matrix-matrix multiplication. If the package *sparse_dot_mkl* is not installed, then the program uses the sparse dot product from Scipy.